<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Queue Management

* The queue is used to task labelers with specific assets
* We can do any of the following:
    * Set quality settings
    * Set the order of items in the queue
    * Set the percent of assets to review

In [15]:
%pip install -q "labelbox[data]"
%pip install -q numpy


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import labelbox as lb
from labelbox.schema.quality_mode import QualityMode
from uuid import uuid4
import json

# API Key and Client
See the developer guide for [creating an API key](https://docs.labelbox.com/reference/create-api-key).

In [17]:
# Add your API key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Set up demo project

#### Create project

In [18]:
# Create Labelbox  project

project = client.create_project(
    name="batch-test-project",
    description="a description",
    quality_mode=QualityMode.
    Benchmark,  # For Consensus projects use quality_mode = QualityMode.Consensus
    media_type=lb.MediaType.Image,
)

dataset = client.create_dataset(name="queue_dataset")

AuthenticationError: Invalid API key('Invalid API key', None)

#### Create ontology and attach to project

In [ ]:
classification_features = [
    lb.Classification(
        class_type=lb.Classification.Type.CHECKLIST,
        name="Quality Issues",
        options=[
            lb.Option(value="blurry", label="Blurry"),
            lb.Option(value="distorted", label="Distorted"),
        ],
    )
]

ontology_builder = lb.OntologyBuilder(
    tools=[], classifications=classification_features
)

ontology = client.create_ontology(
    "Ontology from new features",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

project.setup_editor(ontology)

# Add data to your dataset

In [ ]:
## Example image
uploads = []
global_keys = []
# Generate data rows
for i in range(1, 5):
    global_key = str(uuid4())
    row = {
        "row_data":
            f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key":
            global_key,
    }
    global_keys.append(global_key)
    uploads.append(row)

data_rows = dataset.create_data_rows(uploads)
data_rows.wait_till_done()
print("Errors", data_rows.errors)
print("Dataset status: ", data_rows.status)

# Attach data to your project and set data row priority

In [ ]:
######## Create batches

# Create the batch

batch = project.create_batch(
    "batch-demo",  # Each batch in a project must have a unique name
    global_keys=global_keys[
        0:2],  # A list of data rows, data row ids or global keys
    priority=
    5,  # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

batch2 = project.create_batch(
    "batch-demo-2",  # Each batch in a project must have a unique name
    # Provide a slice of the data since you can't import assets with global keys that already exist in the project.
    global_keys=global_keys[
        2:4],  # A list of data rows, data row ids or global keys
    priority=
    1,  # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

print("Batch: ", batch)
print("Batch2: ", batch2)

In [ ]:
print(
    "View the results here:", f"https://app.labelbox.com/projects/{project.uid}"
)
# Click `start labeling` to see the images in order

## Queue Order
- Add priority for each data row
- Update priority for each data row

In [ ]:
export_task = project.export()
export_task.wait_till_done()

In [ ]:
# Get data rows from project
data_rows = []


def json_stream_handler(output: lb.JsonConverterOutput):
    data_row = json.loads(output.json_str)
    data_rows.append(
        lb.GlobalKey(data_row["data_row"]["global_key"])
    )  # Convert json data row into data row identifier object


if export_task.has_errors():
    export_task.get_stream(
        converter=lb.JsonConverter(), stream_type=lb.StreamType.ERRORS
    ).start(stream_handler=lambda error: print(error))

if export_task.has_result():
    export_json = export_task.get_stream(
        converter=lb.JsonConverter(), stream_type=lb.StreamType.RESULT
    ).start(stream_handler=json_stream_handler)

In [ ]:
# Get label parameter overrides (LPOs)
project_lpos = project.labeling_parameter_overrides()

for lpo in project_lpos:
    print(lpo)
    print("Data row:", lpo.data_row().uid)

In [ ]:
# Add LPOs
lpos = []
priority = 1
for data_row in data_rows:
    lpos.append((data_row, priority))
    priority += 1

project.set_labeling_parameter_overrides(lpos)

# Check results
project_lpos = list(project.labeling_parameter_overrides())

for lpo in project_lpos:
    print(lpo)

In [ ]:
# Update LPOs
global_keys = []
for data_row in data_rows:
    global_keys.append(data_row.key)

project.update_data_row_labeling_priority(
    data_rows=lb.GlobalKeys(global_keys), priority=1
)

# Check results
project_lpos = list(project.labeling_parameter_overrides())

for lpo in project_lpos:
    print(lpo)

# Cleanup

In [ ]:
# project.delete()
# dataset.delete()